# Yandex.Audience (Яндекс-Аудитории)

Базовые функции управления сегментами (используется для управления рекламной кампанией)

In [2]:
import json
import requests

from lib_yaaudience.__init__ import *
from lib_yaaudience.client import *
from lib_yaaudience.core import *
from lib_yaaudience.objects import *

import pandas as pd
import time
import gspread
from oauth2client.service_account import ServiceAccountCredentials

# Вывод всех операций в Jupyter
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
# Чтобы выводить на экран
logger = logging.getLogger()
logger.setLevel(logging.INFO)

### Параметры доступа к Я-Аудитории

In [3]:
# Токен от Яндекс_метрики
token_YM = '******************************************'

headers = {
    "Host": "api-audience.yandex.ru",
    "Authorization": "OAuth " + token_YM,
    "Content-Type": "application/json",
    "Content-Length": "123"
}

# Создание объекта для оберточной API
ya = YaAudience(token=token_YM)

# Удалить сегмент

In [18]:
# https://yandex.ru/dev/audience/doc/segments/delete.html

# идентификатор сегмента
segmentId = 25621611

URL = 'https://api-audience.yandex.ru/v1/management/segment/'
URL = URL + str(segmentId) + "?"

req = requests.delete(URL, headers=headers)
req
req.text

<Response [200]>

'{"success":true}'

### через ya-api

In [10]:
ya.segment_delete(25514140)

True

# *** Вывести список всех сегментов

Статус обработки сегмента:
* uploaded — сегмент с данными пользователя загружен;
* is_processed — сегмент с заданными параметрами обрабатывается;
* processed — загрузка и обработка сегмента завершена. Сегмент готов для работы;
* processing_failed — обработка сегмента завершилась с ошибкой;
* is_updated — сегмент обновляется;
* few_data — сегмент содержит мало данных для работы.

In [25]:
# https://yandex.ru/dev/audience/doc/segments/segments.html

URL = 'https://api-audience.yandex.ru/v1/management/segments'

req = requests.get(URL, headers=headers)
req
req.text

<Response [200]>

'{"segments":[{"id":24286352,"name":"test____segment_geo","type":"geo","status":"processed","create_time":"2022-07-11T16:50:49+03:00","owner":"smbz-demis","has_guests":false,"guest_quantity":0,"can_create_dependent":false,"has_derivatives":false,"cookies_matched_quantity":83520,"geo_segment_type":"last","times_quantity":0,"period_length":0,"form":"circle","radius":1000,"points":[{"latitude":55.812948,"longitude":37.650164,"description":"test_comment"}],"too_much_data":false,"guest":false},{"id":24286564,"name":"test____segment_from_metrika","type":"metrika","status":"processed","create_time":"2022-07-11T16:55:28+03:00","owner":"smbz-demis","has_guests":false,"guest_quantity":0,"can_create_dependent":true,"has_derivatives":false,"cookies_matched_quantity":948,"metrika_segment_type":"goal_id","metrika_segment_id":222601576,"guest":false},{"id":24309942,"name":"test____segment_geo_12","type":"geo","status":"few_data","create_time":"2022-07-12T10:40:02+03:00","owner":"smbz-demis","has_gues

### через ya-api

In [9]:
ya.segments()

[-----------------------------
 
 <SegmentObject{'id': 24396340, 'name': 'test____segment_csv_01', 'type': 'uploading', 'status': 'few_data', 'create_time': datetime.datetime(2022, 7, 14, 15, 59, 34, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'can_create_dependent': False, 'has_derivatives': False, 'cookies_matched_quantity': 2, 'hashed': False, 'content_type': 'crm', 'item_quantity': 188, 'valid_unique_quantity': 188, 'valid_unique_percentage': '100%', 'matched_quantity': 1, 'matched_percentage': '0.53%', 'counter_id': 0, 'guest': False}>
 
 -----------------------------,
 -----------------------------
 
 <SegmentObject{'id': 24733284, 'name': 'test____segment_from_metrika_02', 'type': 'metrika', 'status': 'processed', 'create_time': datetime.datetime(2022, 7, 20, 8, 25, 28, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'c

# reprocess - Пересчёт сегмента

In [78]:
# https://yandex.ru/dev/audience/doc/segments/reprocesssegment.html

# идентификатор сегмента
segmentId = 24286564

URL = f"https://api-audience.yandex.ru/v1/management/segment/{segmentId}/reprocess"

req = requests.put(URL, headers=headers)
req
req.text

<Response [400]>

'{"errors":[{"error_type":"reprocess_wrong_status","message":"Segment 24286564 is in status is_updated and can not be reprocessed"}],"code":400,"message":"Segment 24286564 is in status is_updated and can not be reprocessed"}'

### через ya-api

In [3]:
ya.segment_reprocess(24286352)

True

# create_metrika - Создать сегмент по цели Яндекс-Метрики

In [17]:
# https://yandex.ru/dev/audience/doc/segments/createmetrika.html

metrika_id = 222601576

URL = "https://api-audience.yandex.ru/v1/management/segments/create_metrika?"


body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_from_metrika_02"
body['segment']['metrika_segment_type'] = "goal_id"
body['segment']['metrika_segment_id'] = metrika_id


# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [504]>

'Service unavailable'

### через ya-api

In [21]:
body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_from_metrika_02"
body['segment']['metrika_segment_type'] = "goal_id"
body['segment']['metrika_segment_id'] = 222601576

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False)

ya.segments_create_metrika(body)

-----------------------------

<SegmentObject{'id': 24307274, 'name': 'test____segment_from_metrika_09', 'type': 'metrika', 'status': 'is_processed', 'create_time': datetime.datetime(2022, 7, 12, 9, 53, 46, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'can_create_dependent': False, 'has_derivatives': False, 'cookies_matched_quantity': None, 'hashed': None, 'content_type': None, 'item_quantity': None, 'valid_unique_quantity': None, 'valid_unique_percentage': None, 'matched_quantity': None, 'matched_percentage': None, 'counter_id': None, 'guest': False}>

-----------------------------

# create_geo - Создание сегмента с типом «геолокация-окружности»

In [8]:
# https://yandex.ru/dev/audience/doc/segments/creategeo.html

URL = "https://api-audience.yandex.ru/v1/management/segments/create_geo?"

point_1 = {}
point_1['latitude'] = 55.913569
point_1['longitude'] = 37.573879
point_1['description'] = "Ashan_2"

body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_geo_04"
# Не менее 500 метров, иначе - impossible to create geo segment with radius that is less than 500
body['segment']['radius'] = 500
body['segment']['points'] = [point_1]

# Период посещений указанных мест. Параметр необходим для создания условия «Пользователь посетил указанные места N раз за период».
# Допустимые значения периода (в сутках): от 1 до 90 (включительно).
body['segment']['period_length'] = 10

# Частота посещений указанных мест. Параметр необходим для создания условия «Пользователь посетил указанные места N раз за период».
# Примечание: Учитывается только одно посещение в сутки
body['segment']['times_quantity'] = 1

# Тип координат для создания сегмента.
# last — актуальные координаты. В сегмент попадут пользователи, которые находятся на выбранной территории прямо сейчас, 
#         или находились там не более часа назад.
# regular — регулярные координаты. В сегмент попадут пользователи, которые регулярно бывают на выбранной территории. 
#     Например, живут или работают рядом. При подборе учитываются данные за последние 45 дней.
# home — координаты дома. В сегмент попадут пользователи, которые живут на выбранной территории.
# work — координаты работы. В сегмент попадут пользователи, которые работают на выбранной территории.
# condition — условие «Пользователь посетил указанные места N раз за период». 
#     В сегмент попадут пользователи, которые выполнили заданное условие.
body['segment']['geo_segment_type'] = 'condition'

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [200]>

'{"segment":{"id":25514140,"name":"test____segment_geo_04","type":"geo","status":"is_processed","create_time":"2022-08-05T17:33:21+03:00","owner":"smbz-demis","has_guests":false,"guest_quantity":0,"can_create_dependent":false,"has_derivatives":false,"geo_segment_type":"condition","times_quantity":1,"period_length":10,"form":"circle","radius":500,"points":[{"latitude":55.913569,"longitude":37.573879,"description":"Ashan_2"}],"guest":false}}'

### через ya-api

In [43]:
point_1 = {}
point_1['latitude'] = 55.913569
point_1['longitude'] = 37.573879
point_1['description'] = "Ashan"

body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_geo_04"
body['segment']['radius'] = 350
body['segment']['points'] = [point_1]
body['segment']['geo_segment_type'] = "last"

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False)

ya.segments_create_geo(body)

-----------------------------

<SegmentObject{'id': 24748269, 'name': 'test____segment_geo_04', 'type': 'geo', 'status': 'is_processed', 'create_time': datetime.datetime(2022, 7, 20, 13, 23, 28, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'can_create_dependent': False, 'has_derivatives': False, 'cookies_matched_quantity': None, 'hashed': None, 'content_type': None, 'item_quantity': None, 'valid_unique_quantity': None, 'valid_unique_percentage': None, 'matched_quantity': None, 'matched_percentage': None, 'counter_id': None, 'guest': False}>

-----------------------------

## create_geo_polygon - Создание сегмента с типом «геолокация-полигоны»

In [4]:
# https://yandex.ru/dev/audience/doc/segments/creategeopolygon.html

# В качестве примера Ашан+ЛеруаМерлен:
    # https://yandex.ru/maps/1/moscow-and-moscow-oblast/geo/torgovo_promyshlennaya_zona_altufyevo/1965322886/?indoorLevel=1&ll=37.573310%2C55.913935&mode=search&sctx=ZAAAAAgBEAAaKAoSCa%2FOMSB7s0JAEYyjchO17EtAEhIJ1lOrr64K5T8Rt5ifG5qyyz8iBgABAgMEBSgKOABApIIGSAFiK3JlYXJyPXNjaGVtZV9Mb2NhbC9HZW8vRW5hYmxlQmVhdXR5RmlsdGVyPTFqAnJ1nQHNzEw9oAEAqAEAvQFmLpdowgEGro297osD6gEA8gEA%2BAEAggII0LDRiNCw0L2KAgCSAgCaAgxkZXNrdG9wLW1hcHM%3D&sll=37.573310%2C55.913935&sspn=0.013557%2C0.004454&text=%D0%B0%D1%88%D0%B0%D0%BD&z=16.62

URL = "https://api-audience.yandex.ru/v1/management/segments/create_geo_polygon"

# Задаем 5 координаты ПОЛИГОНА
point_1 = {}
point_1['latitude'] = 55.913832
point_1['longitude'] = 37.575961
point_1['description'] = "point_1"

point_2 = {}
point_2['latitude'] = 55.916601
point_2['longitude'] = 37.572339
point_2['description'] = "point_2"

point_3 = {}
point_3['latitude'] = 55.913379
point_3['longitude'] = 37.568325
point_3['description'] = "point_3"

point_4 = {}
point_4['latitude'] = 55.911825
point_4['longitude'] = 37.570004
point_4['description'] = "point_4"

# Строим один полигон (может быть 10)
polygon_1 = {}
polygon_1['points'] = [point_1, point_2, point_3, point_4, point_1]

# Тело запроса
body = {}
body['segment'] = {}
body['segment']['name'] = "test____Ashan_Lerua"
# В списке - не более 10 полигонов
body['segment']['polygons'] = [polygon_1]

# Период посещений указанных мест. Параметр необходим для создания условия «Пользователь посетил указанные места N раз за период».
# Допустимые значения периода (в сутках): от 1 до 90 (включительно).
body['segment']['period_length'] = 10

# Частота посещений указанных мест. Параметр необходим для создания условия «Пользователь посетил указанные места N раз за период».
# Примечание: Учитывается только одно посещение в сутки
body['segment']['times_quantity'] = 1

# Тип координат для создания сегмента.
# last — актуальные координаты. В сегмент попадут пользователи, которые находятся на выбранной территории прямо сейчас, 
#         или находились там не более часа назад.
# regular — регулярные координаты. В сегмент попадут пользователи, которые регулярно бывают на выбранной территории. 
#     Например, живут или работают рядом. При подборе учитываются данные за последние 45 дней.
# home — координаты дома. В сегмент попадут пользователи, которые живут на выбранной территории.
# work — координаты работы. В сегмент попадут пользователи, которые работают на выбранной территории.
# condition — условие «Пользователь посетил указанные места N раз за период». 
#     В сегмент попадут пользователи, которые выполнили заданное условие.
body['segment']['geo_segment_type'] = 'condition'

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [200]>

'{"segment":{"id":25148605,"name":"test____Ashan_Lerua","type":"geo","status":"is_processed","create_time":"2022-07-28T09:00:24+03:00","owner":"smbz-demis","has_guests":false,"guest_quantity":0,"can_create_dependent":false,"has_derivatives":false,"geo_segment_type":"condition","times_quantity":1,"period_length":10,"form":"polygon","polygons":[{"points":[{"latitude":55.913832,"longitude":37.575961},{"latitude":55.916601,"longitude":37.572339},{"latitude":55.913379,"longitude":37.568325},{"latitude":55.911825,"longitude":37.570004},{"latitude":55.913832,"longitude":37.575961}]}],"guest":false}}'

### через ya-api

In [3]:
point_1 = {}
point_1['latitude'] = 67.812948
point_1['longitude'] = 49.650164
point_1['description'] = "test_comment_02"

body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_geo_03"
body['segment']['radius'] = 2000
body['segment']['points'] = [point_1]
body['segment']['geo_segment_type'] = "last"

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False)

ya.segments_create_geo(body)

-----------------------------

<SegmentObject{'id': 24309942, 'name': 'test____segment_geo_12', 'type': 'geo', 'status': 'is_processed', 'create_time': datetime.datetime(2022, 7, 12, 10, 40, 2, tzinfo=datetime.timezone(datetime.timedelta(seconds=10800))), 'owner': 'smbz-demis', 'has_guests': False, 'guest_quantity': 0, 'can_create_dependent': False, 'has_derivatives': False, 'cookies_matched_quantity': None, 'hashed': None, 'content_type': None, 'item_quantity': None, 'valid_unique_quantity': None, 'valid_unique_percentage': None, 'matched_quantity': None, 'matched_percentage': None, 'counter_id': None, 'guest': False}>

-----------------------------

# upload_csv_file - Создать сегмент из CSV-файла

Создает сегмент из файла с данными. В файле должно быть не менее 100 записей

In [4]:
# https://yandex.ru/dev/audience/doc/segments/uploadcsvfile.html
file_name = "csv/clusters.csv"

URL = "https://api-audience.yandex.ru/v1/management/segments/upload_csv_file"
headers = {"Authorization": f"OAuth {token_YM}"}
file = open(file_name,"r").read()
body = {'file': file} 

req = requests.post(URL, files=body, headers=headers).json()
req

{'segment': {'id': 25319859,
  'type': 'uploading',
  'status': 'uploaded',
  'has_guests': False,
  'guest_quantity': 0,
  'can_create_dependent': False,
  'has_derivatives': False,
  'hashed': False,
  'item_quantity': 4715,
  'guest': False}}

# вар_1 Сохранения сегмента, созданного из файла

In [12]:
# https://yandex.ru/dev/audience/doc/segments/confirm.html

segmentId = 24733341
URL = f"https://api-audience.yandex.ru/v1/management/segment/{segmentId}/confirm"

body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_csv_04"

# Является ли захешированной каждая строка загруженного файла.
# Возможные значения: 0 — строка не захеширована; 1 — строка захеширована.
body['segment']['hashed'] = 0

# Вид содержимого файла.
# Возможные значения: idfa_gaid — идентификаторы устройств; mac — MAC-адреса; crm — CRM-данные.
body['segment']['content_type'] = "crm" 

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [504]>

'Service unavailable'

# вар_2 Сохранения сегмента, созданного из файла (из ClientId Метрики)

In [5]:
# https://yandex.ru/dev/audience/doc/segments/confirmclientid.html

# id-счетчика (в Яндекс-Метрике)
counterId = 65071099

segmentId = 25319859
URL = f"https://api-audience.yandex.ru/v1/management/segment/client_id/{segmentId}/confirm"

body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_csv_07__4715"

body['segment']['counter_id'] = counterId


# Является ли захешированной каждая строка загруженного файла.
# Возможные значения: 0 — строка не захеширована; 1 — строка захеширована.
body['segment']['hashed'] = 0

# Вид содержимого файла.
# Возможные значения: idfa_gaid — идентификаторы устройств; mac — MAC-адреса; crm — CRM-данные.
body['segment']['content_type'] = "crm" 

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [200]>

'{"segment":{"id":25319859,"name":"test____segment_csv_07__4715","type":"uploading","status":"is_processed","create_time":"2022-08-01T23:30:48+03:00","owner":"smbz-demis","has_guests":false,"guest_quantity":0,"can_create_dependent":false,"has_derivatives":false,"hashed":false,"content_type":"client_id","item_quantity":4715,"valid_unique_quantity":4715,"valid_unique_percentage":"100%","counter_id":65071099,"guest":false}}'

# create_lookalike - Создание сегмента с типом «lookalike» (похожесть)

В похожий сегмент попадут те у кого похожи на пользователей исходного сегмента:
* Посещаемые сайты
* Поисковые запросы
* Соц-Дем профиль

Если мы хотим расширить аудиторию, то:
* Не рассматриваем распределение по типу устройств (maintain_device_distribution = False)
* Не рассматриваем распределение по GEO (maintain_geo_distribution = False)
* Уменьшаем степень схожести

In [30]:
# https://yandex.ru/dev/audience/doc/segments/createlookalike.html
# https://www.youtube.com/watch?v=1TPvkIeWIdY&t=47s

URL = "https://api-audience.yandex.ru/v1/management/segments/create_lookalike"

# Идентификатор сегмента, на который должен быть похож созданный сегмент
lookalike_link = 24396340

# Степень «похожести». Принимает значения 1 2 3 4 5 (5 - максимальная похожесть (?))
lookalike_value = 3

# Распределение пользователей по типу устройств. Допустимые значения:
# true — распределение пользователей по типу устройств сохраняется, если это возможно;
# false — распределение пользователей по типу устройств не сохраняется.
# Значение по умолчанию: true.
maintain_device_distribution = True

# Распределение пользователей по городу. Допустимые значения:
# true — распределение пользователей по городу сохраняется, если это возможно;
# false — распределение пользователей по городу не сохраняется.
# Значение по умолчанию: true.
maintain_geo_distribution = True


body = {}
body['segment'] = {}
body['segment']['name'] = "test____segment_lookalike_01"
body['segment']['lookalike_link'] = lookalike_link
body['segment']['lookalike_value'] = lookalike_value
# body['segment']['maintain_device_distribution'] = maintain_device_distribution
# body['segment']['maintain_geo_distribution'] = maintain_geo_distribution

# конвертируем словарь в JSON-формат и перекодируем в UTF-8
body = json.dumps(body, ensure_ascii=False).encode('utf8')

req = requests.post(URL, body, headers=headers)
req
req.text

<Response [400]>

'{"errors":[{"error_type":"invalid_parameter","message":"Source segment 24396340 must be processed but is not"}],"code":400,"message":"Source segment 24396340 must be processed but is not"}'